In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from urllib.parse import quote
import random
import re

# 기업 리스트
companies = ["STX엔진"]

# 날짜 리스트 (2024년 전체)
dates = pd.date_range(start="2024-01-01", end="2024-12-31").strftime("%Y.%m.%d").tolist()

# 결과 저장용
results = []
seen = set()

# Chrome 옵션 설정
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# 드라이버 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 크롤링 시작
for company in companies:
    print(f"\n🔍 [기업 시작] {company}")
    encoded_query = quote(company)

    for target_date in dates:
        print(f"   📅 [날짜 처리 중] {target_date}")
        display_date = target_date.replace(".", "")
        count_per_day = 0

        for start in range(1, 1000, 10):
            url = (
                f"https://search.naver.com/search.naver?where=news&query={encoded_query}"
                f"&sort=0&ds={target_date}&de={target_date}"
                f"&nso=so:r,p:from{display_date}to{display_date},a:all"
                f"&start={start}"
            )

            driver.get(url)
            time.sleep(random.uniform(2, 4))  # 페이지 로딩 대기

            try:
                news_elements = driver.find_elements(By.CSS_SELECTOR, "a.n6AJosQA40hUOAe_Vplg.cdv6mdm2_kpW2D6slkm6")
            except Exception as e:
                print(f"      ⚠️ 구조 탐색 실패 - {company} / {target_date} - {e}")
                break

            if not news_elements:
                break  # 뉴스 없으면 다음 날짜로

            for elem in news_elements:
                try:
                    url = elem.get_attribute("href")

                    # 내부 <span> 요소 탐색 (정확한 뉴스 제목 포함)
                    try:
                        span = elem.find_element(By.CSS_SELECTOR, "span.sds-comps-text-type-headline1")
                        raw_title = span.get_attribute("textContent").strip()
                    except:
                        continue  # 제목 span 없으면 skip

                    # 대괄호 [텍스트] 제거
                    cleaned_title = re.sub(r'\[.*?\]', '', raw_title).strip()

                    # 기업명 제거
                    cleaned_title = cleaned_title.replace(company, "").strip(", ").strip()

                    if not url or not cleaned_title:
                        continue

                    if (company, cleaned_title, url) in seen:
                        continue
                    seen.add((company, cleaned_title, url))

                    results.append({
                        "기업명": company,
                        "날짜": target_date,
                        "뉴스제목": cleaned_title,
                        "URL": url
                    })
                    count_per_day += 1
                except Exception as e:
                    print(f"      ⚠️ 추출 오류: {e}")
                    continue

        if count_per_day > 0:
            print(f"   ✅ [수집 완료] {target_date} - {count_per_day}건")

        time.sleep(random.uniform(1, 2))  # 날짜 간 대기

    print("🌙 [대기] 다음 기업 전환 전 10초 대기...")
    time.sleep(10)

# 드라이버 종료
driver.quit()

# 저장
output_filename = f"{companies[0]}_2024년_뉴스.csv"
df = pd.DataFrame(results)
df.to_csv(output_filename, index=False, encoding='utf-8-sig')
print(f"\n📁 전체 완료! 총 {len(df)}건의 뉴스가 저장되었습니다: {output_filename}")


🔍 [기업 시작] STX엔진
   📅 [날짜 처리 중] 2024.01.01
   ✅ [수집 완료] 2024.01.01 - 3건
   📅 [날짜 처리 중] 2024.01.02
   📅 [날짜 처리 중] 2024.01.03
   ✅ [수집 완료] 2024.01.03 - 9건
   📅 [날짜 처리 중] 2024.01.04
   ✅ [수집 완료] 2024.01.04 - 3건
   📅 [날짜 처리 중] 2024.01.05
   ✅ [수집 완료] 2024.01.05 - 2건
   📅 [날짜 처리 중] 2024.01.06
   📅 [날짜 처리 중] 2024.01.07
   📅 [날짜 처리 중] 2024.01.08
   ✅ [수집 완료] 2024.01.08 - 8건
   📅 [날짜 처리 중] 2024.01.09
   ✅ [수집 완료] 2024.01.09 - 4건
   📅 [날짜 처리 중] 2024.01.10
   ✅ [수집 완료] 2024.01.10 - 4건
   📅 [날짜 처리 중] 2024.01.11
   ✅ [수집 완료] 2024.01.11 - 1건
   📅 [날짜 처리 중] 2024.01.12
   ✅ [수집 완료] 2024.01.12 - 3건
   📅 [날짜 처리 중] 2024.01.13
   📅 [날짜 처리 중] 2024.01.14
   ✅ [수집 완료] 2024.01.14 - 6건
   📅 [날짜 처리 중] 2024.01.15
   ✅ [수집 완료] 2024.01.15 - 2건
   📅 [날짜 처리 중] 2024.01.16
   ✅ [수집 완료] 2024.01.16 - 1건
   📅 [날짜 처리 중] 2024.01.17
   ✅ [수집 완료] 2024.01.17 - 3건
   📅 [날짜 처리 중] 2024.01.18
   ✅ [수집 완료] 2024.01.18 - 3건
   📅 [날짜 처리 중] 2024.01.19
   ✅ [수집 완료] 2024.01.19 - 2건
   📅 [날짜 처리 중] 2024.01.20
   ✅ [수집 완료] 2024.01.20 - 1건